In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv('./total_data.csv')
monthly = pd.read_csv('./month_stock.csv')
stocks = pd.read_csv('./trade_train_set/stocks.csv')
trade_train = pd.read_csv('./trade_train_set/trade_train.csv')

In [3]:
len(data[data['20년7월TOP3대상여부']=='Y']['종목번호'].unique())

133

In [4]:
stocks = stocks.drop(['index'], axis=1)
trade_train = trade_train.drop(['Unnamed: 0'], axis=1)

In [5]:
'''****** 변수 수정

거래량, 가격 -> 1을 기준으로 scaling, or 전월을 기준으로 +a%
20년도 7월에 포함되지 않은 것은 모두 drop


****** 확실히 버릴 것
그룹내_매수여부
그룹내_매도여부
기준년월
평균매수수량
평균매도수량
매수가격_중앙값
매도가격_중앙값
20년 7월 top3
종목명


******* 애매한 것
매도고객수 (전월 데이터를 사용할까?) -> 사용한다면 19년 7월은 어떻게 할까?

산업 대분류

산업 중분류

산업 소분류



******* 가져갈 것.

그룹번호
그룹내 고객수
종목번호
매수고객수 (Y)
거래량
거래금액_만원단위(정규화)
시장구분(정규화)
평균가격(정규화)
직전달 TOP3 여부'''

'****** 변수 수정\n\n거래량, 가격 -> 1을 기준으로 scaling, or 전월을 기준으로 +a%\n20년도 7월에 포함되지 않은 것은 모두 drop\n\n\n****** 확실히 버릴 것\n그룹내_매수여부\n그룹내_매도여부\n기준년월\n평균매수수량\n평균매도수량\n매수가격_중앙값\n매도가격_중앙값\n20년 7월 top3\n종목명\n\n\n******* 애매한 것\n매도고객수 (전월 데이터를 사용할까?) -> 사용한다면 19년 7월은 어떻게 할까?\n\n산업 대분류\n\n산업 중분류\n\n산업 소분류\n\n\n\n******* 가져갈 것.\n\n그룹번호\n그룹내 고객수\n종목번호\n매수고객수 (Y)\n거래량\n거래금액_만원단위(정규화)\n시장구분(정규화)\n평균가격(정규화)\n직전달 TOP3 여부'

In [6]:
data = data[data['20년7월TOP3대상여부'] == 'Y']

data = data.drop(['그룹내_매수여부', '그룹내_매도여부', '평균매수수량', '평균매도수량', '매수가격_중앙값', '매도가격_중앙값', 
          '20년7월TOP3대상여부', '종목명'], axis=1)

In [7]:
dat_mean = data[['거래금액_만원단위', '평균가격', '거래량']].mean(axis=0)

dat_std = data[['거래금액_만원단위', '평균가격', '거래량']].std()

from sklearn.preprocessing import StandardScaler
#data[['거래금액_만원단위', '평균가격', '거래량']] = StandardScaler().fit_transform(data[['거래금액_만원단위', '평균가격', '거래량']])

data

,기준년월,그룹번호,그룹내고객수,종목번호,매수고객수,매도고객수,거래량,거래금액_만원단위,시장구분,표준산업구분코드_대분류,표준산업구분코드_소분류,표준산업구분코드_중분류,평균가격
0,201907,MAD01,288,A000660,7,17,2.993388e+06,2.244919e+07,코스피,제조업,반도체 제조업,전자부품· 컴퓨터· 영상· 음향 및,74995.933792
1,201907,MAD04,776,A000660,23,36,2.993388e+06,2.244919e+07,코스피,제조업,반도체 제조업,전자부품· 컴퓨터· 영상· 음향 및,74995.933792
2,201907,MAD05,890,A000660,3,11,2.993388e+06,2.244919e+07,코스피,제조업,반도체 제조업,전자부품· 컴퓨터· 영상· 음향 및,74995.933792
3,201907,MAD06,1615,A000660,27,63,2.993388e+06,2.244919e+07,코스피,제조업,반도체 제조업,전자부품· 컴퓨터· 영상· 음향 및,74995.933792
4,201907,MAD09,634,A000660,7,13,2.993388e+06,2.244919e+07,코스피,제조업,반도체 제조업,전자부품· 컴퓨터· 영상· 음향 및,74995.933792
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30162,202006,MAD43,2879,A009240,6,3,1.061898e+05,9.436800e+05,코스피,도매 및 소매업,생활용품 도매업,도매 및 상품 중개업,88867.277123
30164,202006,MAD43,2879,A011150,0,3,1.635518e+06,6.335980e+05,코스피,제조업,기타 식품 제조업,식료품 제조업,3873.990846
30167,202006,MAD43,2879,A036930,3,5,5.442303e+05,3.067193e+05,코스닥,제조업,특수 목적용 기계 제조업,기타 기계 및 장비 제조업,5635.835831
30180,202006,MAD43,2879,A158430,6,5,1.758884e+05,5.464499e+05,코스닥,정보통신업,소프트웨어 개발 및 공급업,출판업,31067.988614


In [8]:
a = trade_train[['기준년월', '그룹번호', '종목번호', '매수고객수']].groupby(['기준년월', '그룹번호','종목번호']).sum(
).sort_values(by = ['기준년월','그룹번호','매수고객수'], ascending=[True, True, False]).reset_index(drop=False)

a = a.groupby(['기준년월', '그룹번호']).head(3)

prev_month_list = list(a[a['기준년월'] == 202006]['종목번호'])

data['직전달TOP3여부'] = 0
for i in range(data.shape[0]):
    if data.iloc[i,3] in prev_month_list:
        data.iloc[i,-1] = 1

In [9]:
sum(data['직전달TOP3여부'] == 1)

4805

In [10]:
len(data[data['직전달TOP3여부'] == 1]['종목번호'].unique())

25

In [11]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
def dummy(data,col):
    lab=LabelEncoder()
    aa=lab.fit_transform(data[col]).reshape(-1,1)
    ohe=OneHotEncoder(sparse=False)
    column_names=[col+'_'+ str(i) for i in lab.classes_]
    return(pd.DataFrame(ohe.fit_transform(aa),columns=column_names))

In [12]:
categ_columns = ['그룹번호', '종목번호','표준산업구분코드_대분류', '표준산업구분코드_중분류', '표준산업구분코드_소분류', '시장구분']
n_columns = ['기준년월','그룹내고객수', '매수고객수', '거래량', '거래금액_만원단위', '평균가격', '직전달TOP3여부']

df = data[n_columns]
df.index = range(df.shape[0])

In [13]:
lab = LabelEncoder()
for col in categ_columns:
    temp_df = pd.DataFrame(lab.fit_transform(data[col]))
    temp_df.columns = [col]
    df = pd.concat([df, temp_df], axis=1)

In [14]:
df.to_csv('./df.csv',mode='w', header=True, index=False)

In [15]:
df

,기준년월,그룹내고객수,매수고객수,거래량,거래금액_만원단위,평균가격,직전달TOP3여부,그룹번호,종목번호,표준산업구분코드_대분류,표준산업구분코드_중분류,표준산업구분코드_소분류,시장구분
0,201907,288,7,2.993388e+06,2.244919e+07,74995.933792,1,0,3,9,26,19,1
1,201907,776,23,2.993388e+06,2.244919e+07,74995.933792,1,3,3,9,26,19,1
2,201907,890,3,2.993388e+06,2.244919e+07,74995.933792,1,4,3,9,26,19,1
3,201907,1615,27,2.993388e+06,2.244919e+07,74995.933792,1,5,3,9,26,19,1
4,201907,634,7,2.993388e+06,2.244919e+07,74995.933792,1,8,3,9,26,19,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7485,202006,2879,6,1.061898e+05,9.436800e+05,88867.277123,0,42,26,4,10,22,1
7486,202006,2879,0,1.635518e+06,6.335980e+05,3873.990846,0,42,32,9,14,12,1
7487,202006,2879,3,5.442303e+05,3.067193e+05,5635.835831,0,42,58,9,7,50,0
7488,202006,2879,6,1.758884e+05,5.464499e+05,31067.988614,0,42,100,8,29,26,0


In [16]:
# 예측 템플릿 만들기
그룹번호 = pd.Series(df['그룹번호'].unique())
종목번호 = pd.Series(df['종목번호'].unique())


temp = []
for i in 그룹번호:
    for j in 종목번호:
        temp.append([i, j])

                                
temp = pd.DataFrame(data=temp, columns = ['그룹번호', '종목번호'])

In [17]:
temp

,그룹번호,종목번호
0,0,3
1,0,12
2,0,16
3,0,17
4,0,18
...,...,...
6379,6,122
6380,6,81
6381,6,46
6382,6,29


In [18]:
a = trade_train[['그룹번호','그룹내고객수']].groupby('그룹번호').first().reset_index(drop=False)
a['그룹번호'] = lab.fit_transform(a['그룹번호'])

temp = pd.merge(temp, a, on='그룹번호')

temp = pd.merge(temp, df[['표준산업구분코드_대분류','표준산업구분코드_중분류', '표준산업구분코드_소분류', '종목번호', '직전달TOP3여부', '시장구분']].drop_duplicates(), on='종목번호')

In [19]:
temp

,그룹번호,종목번호,그룹내고객수,표준산업구분코드_대분류,표준산업구분코드_중분류,표준산업구분코드_소분류,직전달TOP3여부,시장구분
0,0,3,288,9,26,19,1,1
1,3,3,776,9,26,19,1,1
2,4,3,890,9,26,19,1,1
3,5,3,1615,9,26,19,1,1
4,8,3,634,9,26,19,1,1
...,...,...,...,...,...,...,...,...
6379,28,132,115,9,14,17,0,0
6380,44,132,204,9,14,17,0,0
6381,9,132,106,9,14,17,0,0
6382,39,132,82,9,14,17,0,0


In [20]:
a = stocks[stocks['기준일자'] > 20200700].groupby(['종목번호']).sum().reset_index(drop=False)[['종목번호', '거래금액_만원단위']]
a['종목번호'] = lab.fit_transform(a['종목번호'])
b = stocks[stocks['기준일자'] > 20200700].groupby(['종목번호']).sum().reset_index(drop=False)[['종목번호','거래량']]
b['종목번호'] = lab.fit_transform(b['종목번호'])

a = pd.merge(a, b, on='종목번호')
a['평균가격'] = (a['거래금액_만원단위'] / a['거래량']) * 10000
a

,종목번호,거래금액_만원단위,거래량,평균가격
0,0,105851663,50746073,20859.084603
1,1,46342841,10777257,43000.590039
2,2,100267371,18098351,55401.384911
3,3,15857304,995851,159233.700624
4,4,1661316,773330,21482.627080
...,...,...,...,...
1082,1082,41941116,11584571,36204.289309
1083,1083,7632345,3892564,19607.500352
1084,1084,4170002,1067286,39071.083102
1085,1085,7254723,2170002,33431.872413


In [21]:
temp = pd.merge(temp, a, on='종목번호')

#temp['거래금액_만원단위'] = (temp['거래금액_만원단위'] - dat_mean[0]) / dat_std[0]
#temp['평균가격'] = (temp['평균가격'] - dat_mean[1]) / dat_std[1]

temp

,그룹번호,종목번호,그룹내고객수,표준산업구분코드_대분류,표준산업구분코드_중분류,표준산업구분코드_소분류,직전달TOP3여부,시장구분,거래금액_만원단위,거래량,평균가격
0,0,3,288,9,26,19,1,1,15857304,995851,159233.700624
1,3,3,776,9,26,19,1,1,15857304,995851,159233.700624
2,4,3,890,9,26,19,1,1,15857304,995851,159233.700624
3,5,3,1615,9,26,19,1,1,15857304,995851,159233.700624
4,8,3,634,9,26,19,1,1,15857304,995851,159233.700624
...,...,...,...,...,...,...,...,...,...,...,...
6379,28,132,115,9,14,17,0,0,3530387,19112414,1847.169594
6380,44,132,204,9,14,17,0,0,3530387,19112414,1847.169594
6381,9,132,106,9,14,17,0,0,3530387,19112414,1847.169594
6382,39,132,82,9,14,17,0,0,3530387,19112414,1847.169594


In [22]:
df

,기준년월,그룹내고객수,매수고객수,거래량,거래금액_만원단위,평균가격,직전달TOP3여부,그룹번호,종목번호,표준산업구분코드_대분류,표준산업구분코드_중분류,표준산업구분코드_소분류,시장구분
0,201907,288,7,2.993388e+06,2.244919e+07,74995.933792,1,0,3,9,26,19,1
1,201907,776,23,2.993388e+06,2.244919e+07,74995.933792,1,3,3,9,26,19,1
2,201907,890,3,2.993388e+06,2.244919e+07,74995.933792,1,4,3,9,26,19,1
3,201907,1615,27,2.993388e+06,2.244919e+07,74995.933792,1,5,3,9,26,19,1
4,201907,634,7,2.993388e+06,2.244919e+07,74995.933792,1,8,3,9,26,19,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7485,202006,2879,6,1.061898e+05,9.436800e+05,88867.277123,0,42,26,4,10,22,1
7486,202006,2879,0,1.635518e+06,6.335980e+05,3873.990846,0,42,32,9,14,12,1
7487,202006,2879,3,5.442303e+05,3.067193e+05,5635.835831,0,42,58,9,7,50,0
7488,202006,2879,6,1.758884e+05,5.464499e+05,31067.988614,0,42,100,8,29,26,0


In [23]:
temp = temp[['그룹내고객수','거래량','거래금액_만원단위','평균가격','직전달TOP3여부', '그룹번호', 
      '종목번호','표준산업구분코드_대분류', '표준산업구분코드_중분류', '표준산업구분코드_소분류', '시장구분']]

temp.to_csv('./template.csv',mode='w', header=True, index=False)

In [24]:
df.columns = ['date', 'custom_in_grp', 'buyers', 'volume', 'trade_total', 'avg_price', 'top3', 
             'grp', 'stock', 'big', 'mid', 'small', 'market']

df

,date,custom_in_grp,buyers,volume,trade_total,avg_price,top3,grp,stock,big,mid,small,market
0,201907,288,7,2.993388e+06,2.244919e+07,74995.933792,1,0,3,9,26,19,1
1,201907,776,23,2.993388e+06,2.244919e+07,74995.933792,1,3,3,9,26,19,1
2,201907,890,3,2.993388e+06,2.244919e+07,74995.933792,1,4,3,9,26,19,1
3,201907,1615,27,2.993388e+06,2.244919e+07,74995.933792,1,5,3,9,26,19,1
4,201907,634,7,2.993388e+06,2.244919e+07,74995.933792,1,8,3,9,26,19,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7485,202006,2879,6,1.061898e+05,9.436800e+05,88867.277123,0,42,26,4,10,22,1
7486,202006,2879,0,1.635518e+06,6.335980e+05,3873.990846,0,42,32,9,14,12,1
7487,202006,2879,3,5.442303e+05,3.067193e+05,5635.835831,0,42,58,9,7,50,0
7488,202006,2879,6,1.758884e+05,5.464499e+05,31067.988614,0,42,100,8,29,26,0


In [25]:
import lightgbm as lgb

train = df[df['date'] != 202006]
val = df[df['date'] == 202006]

/opt/anaconda3/envs/venv/lib/python3.8/site-packages/lightgbm/__init__.py:42: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  warnings.warn("Starting from version 2.2.1, the library file in distribution wheels for macOS "


In [26]:
x_train = train.drop(['date','buyers'], axis=1)
y_train = train['buyers']
x_val = val.drop(['date','buyers'], axis=1)
y_val = val['buyers']

x_train

,custom_in_grp,volume,trade_total,avg_price,top3,grp,stock,big,mid,small,market
0,288,2.993388e+06,2.244919e+07,74995.933792,1,0,3,9,26,19,1
1,776,2.993388e+06,2.244919e+07,74995.933792,1,3,3,9,26,19,1
2,890,2.993388e+06,2.244919e+07,74995.933792,1,4,3,9,26,19,1
3,1615,2.993388e+06,2.244919e+07,74995.933792,1,5,3,9,26,19,1
4,634,2.993388e+06,2.244919e+07,74995.933792,1,8,3,9,26,19,1
...,...,...,...,...,...,...,...,...,...,...,...
6249,444,4.610566e+06,5.608904e+05,1216.532749,0,41,71,9,1,51,0
6250,2879,3.926797e+05,2.380356e+05,6061.825669,0,42,35,4,10,14,0
6251,2879,2.263296e+05,3.852756e+05,17022.765817,0,42,46,1,25,8,1
6252,2879,4.502947e+05,8.480489e+04,1883.319697,0,42,76,4,10,14,0


In [27]:
categ=['grp', 'big', 'stock', 'mid', 'small']
train_ds = lgb.Dataset(x_train, label=y_train, categorical_feature=categ, free_raw_data=False)
val_ds = lgb.Dataset(x_val, label=y_val, categorical_feature=categ, free_raw_data=False)

In [28]:
2**10

1024

In [29]:
def lgbm_regressor(train_X, train_y, test_X, test_y, feature_names=None, seed_val=2284, num_rounds=3000):

    param = {}

    param['boosting'] = 'gbdt'
    
    param['objective'] = 'regression'

    param['learning_rate'] = 0.05

    param['max_depth'] = 10

    param['metric'] = 'rmse'
    
    param['min_data_in_leaf'] = 100
    
    param['is_training_metric'] = True
    


    param['bagging_fraction'] = 0.8
    
    param['num_leaves'] = 1000

    param['feature_fraction'] = 0.8


    
    param['seed'] = seed_val
    
    num_rounds = num_rounds

    plst = list(param.items())

    model = lgb.train(param, train_ds, num_rounds,val_ds, verbose_eval= 100, early_stopping_rounds=180)

    return model

model = lgbm_regressor(train_X = x_train, train_y = y_train, test_X = x_val, test_y = y_val)

/opt/anaconda3/envs/venv/lib/python3.8/site-packages/lightgbm/basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 180 rounds
[100]	valid_0's rmse: 29.8684
[200]	valid_0's rmse: 26.511
[300]	valid_0's rmse: 24.4287
[400]	valid_0's rmse: 23.2895
[500]	valid_0's rmse: 22.4825
[600]	valid_0's rmse: 21.7872
[700]	valid_0's rmse: 21.2982
[800]	valid_0's rmse: 20.8922
[900]	valid_0's rmse: 20.5198
[1000]	valid_0's rmse: 20.2557
[1100]	valid_0's rmse: 19.9902
[1200]	valid_0's rmse: 19.766
[1300]	valid_0's rmse: 19.5909
[1400]	valid_0's rmse: 19.4084
[1500]	valid_0's rmse: 19.2613
[1600]	valid_0's rmse: 19.0977
[1700]	valid_0's rmse: 18.9822
[1800]	valid_0's rmse: 18.8721
[1900]	valid_0's rmse: 18.7674
[2000]	valid_0's rmse: 18.6637
[2100]	valid_0's rmse: 18.5611
[2200]	valid_0's rmse: 18.4796
[2300]	valid_0's rmse: 18.3727
[2400]	valid_0's rmse: 18.2993
[2500]	valid_0's rmse: 18.2422
[2600]	valid_0's rmse: 18.1834
[2700]	valid_0's rmse: 18.1093
[2800]	valid_0's rmse: 18.0523
[2900]	valid_0's rmse: 18.0133
[3000]	valid_0's rmse: 17.9853
Did not meet early 

In [30]:
temp.columns = x_train.columns

pred = model.predict(temp)

pred

array([ 11.1646091 ,  13.90709824,   6.00567882, ..., -11.93160523,
       -10.19233958, -10.87095603])

In [31]:
a = pd.DataFrame(pred, columns=['buyers'])

result = pd.concat([temp, a], axis = 1)

result

,custom_in_grp,volume,trade_total,avg_price,top3,grp,stock,big,mid,small,market,buyers
0,288,995851,15857304,159233.700624,1,0,3,9,26,19,1,11.164609
1,776,995851,15857304,159233.700624,1,3,3,9,26,19,1,13.907098
2,890,995851,15857304,159233.700624,1,4,3,9,26,19,1,6.005679
3,1615,995851,15857304,159233.700624,1,5,3,9,26,19,1,41.304809
4,634,995851,15857304,159233.700624,1,8,3,9,26,19,1,10.102694
...,...,...,...,...,...,...,...,...,...,...,...,...
6379,115,19112414,3530387,1847.169594,0,28,132,9,14,17,0,-8.718611
6380,204,19112414,3530387,1847.169594,0,44,132,9,14,17,0,-9.029080
6381,106,19112414,3530387,1847.169594,0,9,132,9,14,17,0,-11.931605
6382,82,19112414,3530387,1847.169594,0,39,132,9,14,17,0,-10.192340


In [32]:
tmp = result[['grp', 'stock', 'buyers']].sort_values(by = ['grp','buyers'], ascending=[True, False])

tmp  = tmp.groupby(['grp']).head(3)
tmp[:-100]

,grp,stock,buyers
4512,0,70,137.622799
1152,0,31,137.088459
240,0,19,135.540044
4545,1,70,52.233444
1185,1,31,52.145305
609,1,0,50.883444
1187,2,31,57.742092
4547,2,70,57.741277
611,2,0,56.169608
4513,3,70,196.045761


In [33]:
pd.Series(pred)[:20]

0     11.164609
1     13.907098
2      6.005679
3     41.304809
4     10.102694
5      2.493807
6      2.038869
7      3.256093
8      5.084691
9     10.344698
10     3.796472
11     8.219766
12    51.802382
13     3.830234
14     5.869663
15     7.011761
16    10.686581
17     5.362846
18     6.457070
19     7.501198
dtype: float64

In [34]:
y_val[:20]

6254     62
6255      7
6256      5
6257     73
6258     24
6259    147
6260      9
6261      6
6262     24
6263      4
6264     22
6265      3
6266     20
6267     14
6268      3
6269     39
6270     67
6271     20
6272     10
6273     26
Name: buyers, dtype: int64

In [35]:
lab1 = LabelEncoder()
lab1.fit(data['그룹번호'])
lab2 = LabelEncoder()
lab2.fit(data['종목번호'])

LabelEncoder()

In [36]:
print(lab2.inverse_transform([123]))
print(lab1.inverse_transform([0]))

['A272210']
['MAD01']


In [37]:
0.01*(0.99**70)

0.00494838659600207

In [38]:
66150/350

189.0